In [1]:
import numpy as np
import json 
import pandas as pd 
from scipy.special import gamma, kv
from tqdm import tqdm
import torch
import os

# 1. Read Data, Convert to DataFrame

In [2]:
def convert_json_to_df(json_file_path:str):
    with open(json_file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    in_sample_transactions = data["transactions"]["in_sample_transactions"]
    out_sample_transactions = data["transactions"]["out_of_sample_transactions"]
    product_labels = data['product_labels']

    in_sample_transactions = pd.DataFrame(in_sample_transactions)
    out_sample_transactions = pd.DataFrame(out_sample_transactions)
    
    # rename 'prodcut' to 'choice' 
    in_sample_transactions.rename(columns={'product':'choice'}, inplace=True)
    out_sample_transactions.rename(columns={'product':'choice'}, inplace=True)
    product_labels = pd.DataFrame(
        list(product_labels.items()), columns=["product_id", "product_name"]
    )
    return in_sample_transactions, out_sample_transactions,product_labels

In [3]:
def convert_list_to_one_hot(transaction:list,d):
    one_hot = np.zeros(d)
    for item in transaction:
        one_hot[item] = 1
    return one_hot

def convert_to_one_hot(transactions:pd.DataFrame,d):
    transactions["offered_product_one_hot"] = transactions['offered_products'].apply(lambda x : convert_list_to_one_hot(x,d))
    transactions['choice_one_hot'] = transactions['choice'].apply(lambda x: convert_list_to_one_hot([x],d))
    return transactions



In [4]:
instance_id = 1
in_sample_transactions, out_sample_transactions ,items= convert_json_to_df(f"hotel_json/instance_{instance_id}.json")
d = len(items) + 1 # consider the 0 as no-purchase
datasize = len(in_sample_transactions)
in_sample_transactions = convert_to_one_hot(in_sample_transactions,d)
out_sample_transactions = convert_to_one_hot(out_sample_transactions,d)

in_sample_transactions

,offered_products,choice,offered_product_one_hot,choice_one_hot
0,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10]",6,"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
1,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10]",0,"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10]",0,"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10]",0,"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10]",0,"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
5285,"[0, 1, 3, 4, 6, 7, 8, 9, 10]",1,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5286,"[0, 1, 3, 4, 6, 7, 8, 9, 10]",0,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5287,"[0, 1, 3, 4, 6, 7, 8, 9, 10]",0,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5288,"[0, 1, 3, 4, 6, 7, 8, 9, 10]",0,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# 2. Kernel Implementation





Matrix-valued **Matern kernel** $\tilde{\boldsymbol{\mathsf{k}}}$ can be 

$$
\boldsymbol{\mathsf{k}}(\boldsymbol{e}_{S},\boldsymbol{e}_{S'}) = \boldsymbol{K} \otimes \mathsf{k}_{m}(\boldsymbol{e}_{S}, \boldsymbol{e}_{S'})
$$
where $\boldsymbol{K}$ is a positive semi-definite matrix. Then 

$$
\tilde{\mathsf{k}}^{ij}(\boldsymbol{e}_{S},\boldsymbol{e}_{S'}) = K_{ij} \times\sigma^{2} \frac{2^{1-\nu}}{\Gamma(\nu)} \left( \sqrt{ 2\nu }  \frac{\left\| \boldsymbol{e}_{S} - \boldsymbol{e}_{S'} \right\|_{2}  }{\ell} \right) K_{\nu} \left( \sqrt{ 2\nu } \frac{\left\| \boldsymbol{\boldsymbol{e}_{S}-\boldsymbol{e}_{S'}} \right\|_{2}  }{\ell} \right)
$$
Add constraint, 

$$
\mathsf{k}^{ij}(S, S') = \mathbb{1}(i \in \boldsymbol{e}_{S})\cdot \mathbb{1}(j \in \boldsymbol{e}_{S'}) \cdot   \tilde{\mathsf{k}}^{ij}(\boldsymbol{e}_{S}, \boldsymbol{e}_{S'})
$$


Same with Gaussian kernel.



In [5]:
K = torch.eye(d) # for the moment we consider the identity matrix as the covariance matrix

In [6]:
length_scale=1 
nu= 1
sigma = 1


def generate_scalar_matern_kernel(length_scale:float, nu:float, sigma:float):
    """
    Input: kernel parameters and index (i,j)
    generate base scalar-valued Matern kernel at (i,j) 
    return {k_m}_ij
    """
    def kernel(x1:torch.Tensor, x2:torch.Tensor):
        dist = torch.norm(x1 - x2)
        
        if dist == 0:
            return sigma**2

        # calculate the factor
        factor = (2 ** (1 - nu)) / gamma(nu)
        scaled_dist = np.sqrt(2 * nu) * dist / length_scale
        result = sigma**2 * factor * (scaled_dist**nu) * kv(nu, scaled_dist)

        return result
    
    return kernel

# def generate_scalar_gaussian_kernel(length_scale:float, sigma:float):
#     """
#     Input: kernel parameters and index (i,j)
#     generate base scalar-valued Gaussian kernel at (i,j)
#     return {k_g}_ij
#     """
#     def kernel(x1:np.ndarray, x2:np.ndarray):
        
#         dist = np.linalg.norm(x1 - x2)
#         return sigma**2 * np.exp(-dist**2 / (2 * length_scale**2))
#     return kernel

def generate_matrix_matern_kernel(length_scale, nu, sigma):
    scalar_kernel = generate_scalar_matern_kernel(length_scale, nu, sigma)
    
    def kernel(x1:torch.Tensor, x2:torch.Tensor):
        scalar_kernel_value = scalar_kernel(x1, x2)
        
        mask_x1 = x1 != 0
        mask_x2 = x2 != 0
        
        mask_x1_expand = mask_x1.view(-1, 1).expand(d, d) 
        mask_x2_expand = mask_x2.view(1, -1).expand(d, d)
        
        result = scalar_kernel_value * K* mask_x1_expand * mask_x2_expand 

        return result 
    return kernel




# def generate_matrix_gaussian_kernel(length_scale, sigma):
#     """
#     generate a matrix-valued Gaussian kernel 
#     """
#     scalar_gaussian_kernel = generate_scalar_gaussian_kernel(length_scale, sigma)
#     def kernel(x1:np.ndarray,x2:np.ndarray):
#         dim = x1.shape[0]
#         result = np.zeros((dim,dim))
#         for i in range(dim):
#             for j in range(dim):
#                 if x1[i] == 0 or x2[j] == 0:
#                     result[i,j] = 0
#                 else:
#                     result[i,j] = scalar_gaussian_kernel(x1, x2) * K[i,j]
#         return result
#     return kernel

In [7]:
matrix_matern_kernel = generate_matrix_matern_kernel(length_scale, nu, sigma)

# 3. Solve

## 3.1. Precalcualte Kernel Tensor

In [8]:



total_iterations = datasize * datasize
with tqdm(total=total_iterations, desc="Overall Progress", unit="iteration") as pbar:
    K_kernel = torch.zeros(datasize,datasize,d,d)

    for i in range(datasize):
        for j in range(datasize):
            S_i = torch.tensor(in_sample_transactions["offered_product_one_hot"][i],dtype=float)
            S_j = torch.tensor(in_sample_transactions["offered_product_one_hot"][j],dtype=float)
            K_kernel[i, j] = matrix_matern_kernel(
                S_i,
                S_j,
            )

            pbar.update(1)  

Overall Progress: 100%|██████████| 27984100/27984100 [54:09<00:00, 8611.24iteration/s]  


RuntimeError: Parent directory results/feature_free/hotel_1/kernel_data does not exist.

In [10]:
import os
kernel_data_save_path = f"results/feature_free/hotel_{instance_id}/kernel_data/Matern({length_scale},{nu},{sigma})_kernel_data.pt"
os.makedirs(os.path.dirname(kernel_data_save_path), exist_ok=True)
torch.save(K_kernel, f"results/feature_free/hotel_{instance_id}/kernel_data/Matern({length_scale},{nu},{sigma})_kernel_data.pt")

## 3.2 Solve Using `torch.optimin.Adam`

In [ ]:
K_kernel = torch.load(f"results/feature_free/hotel_{instance_id}/kernel_data/Matern({length_scale},{nu},{sigma})_kernel_data.pt")

In [13]:
alphaset = torch.randn((datasize, d), dtype=torch.float32, requires_grad=True)
lambda_ = 0.01

def objective(alphaset: torch.Tensor):
    U = torch.zeros((datasize, d), dtype=torch.float32)
    U = torch.einsum("ijab, jb -> ia", K_kernel, alphaset)

    l = loss(U)
    r = reg(alphaset)

    return l + lambda_ * r



def loss(U: torch.Tensor):

    loss_value = 0.0
    for i in range(datasize):

        p_vec = torch.zeros((d, 1), dtype=torch.float32)

        hS_i = torch.tensor(
            in_sample_transactions.iloc[i]["offered_product_one_hot"],
            dtype=torch.float32,
        ).view(-1, 1)

        y_i = torch.tensor(
            in_sample_transactions.iloc[i]["choice_one_hot"], dtype=torch.float32
        ).view(-1, 1)

        utility_hSi = U[i].view(-1, 1)
        exp_utility = torch.exp(utility_hSi)
        sum_exp_utility = torch.sum(exp_utility) 

        for j in range(d):

            if hS_i[j] == 1:

                p_vec[j] = torch.exp(utility_hSi[j]) / sum_exp_utility
            else:

                p_vec[j] = 0

        loss_value += cross_entropy_loss(p_vec, y_i)
    return loss_value / datasize


def cross_entropy_loss(p_vec: torch.Tensor, y_vec: torch.Tensor):
    for i in range(d):
        if y_vec[i] == 1:
            return -torch.log(p_vec[i])


def squared_loss(p_vec: torch.Tensor, y_vec: torch.Tensor):
    return torch.sum((p_vec - y_vec) ** 2)


def reg(alphaset: torch.Tensor):

    alphaset = alphaset.unsqueeze(2)  # add a dimension 

    # einsum
    result = torch.einsum("ikd,ijkl,jle->", alphaset, K_kernel, alphaset)

    return result

def compute_gradient():
    objective_value = objective(alphaset)
    objective_value.backward() 
    return alphaset.grad

optimizer = torch.optim.Adam([alphaset], lr=0.01)
for epoch in range(500):
    optimizer.zero_grad()  
    loss_value = objective(alphaset)
    loss_value.backward()  
    optimizer.step()  
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss_value.item()}")



RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 13544304400 bytes.

In [ ]:
obj = objective(alphaset)

In [ ]:
def cal_utility(S:torch.Tensor):
    utility_vec = torch.zeros((d,1),dtype=torch.float32)
    for i in range(datasize):
        S_i = torch.tensor(in_sample_transactions["offered_product_one_hot"][i],dtype=float)
        kernel_value = matrix_matern_kernel(S, S_i)
        utility_vec += kernel_value @ alphaset[i].view(-1,1)
    return utility_vec

def cal_probability(S:np.ndarray):
    utility_vec = cal_utility(S)
    sum_exp_utility = torch.sum(torch.exp(utility_vec))
    p_vec = torch.zeros(d,dtype=torch.float32)
    for i in range(d):
        if S[i] == 1:
           p_vec[i] = torch.exp(utility_vec[i]) / sum_exp_utility 
        else:
            p_vec[i] = 0
            
    return p_vec
    



S = torch.tensor([1,1,1,1,1,1,1],dtype=float)
cal_probability(S)

tensor([0.7527, 0.0795, 0.0376, 0.0248, 0.0277, 0.0536, 0.0242],
       grad_fn=<CopySlices>)

# 4. Performance

In [ ]:

S_test = torch.stack([torch.tensor(sample, dtype=torch.float32) for sample in out_sample_transactions["offered_product_one_hot"]])
p_vecs = torch.stack([cal_probability(S) for S in S_test])
choice_vecs = torch.stack([torch.tensor(sample, dtype=torch.float32) for sample in out_sample_transactions["choice_one_hot"]])


In [ ]:
p_vecs.shape, choice_vecs.shape

(torch.Size([255, 7]), torch.Size([255, 7]))

In [ ]:
mse = torch.mean((p_vecs- choice_vecs) ** 2).item()
rmse = np.sqrt(mse)
rmse,mse

(0.232182836626952, 0.05390886962413788)

In [ ]:
# Save results
import csv
results = [
    [float(obj.detach()),mse, rmse, ]
]
with open(f'results/feature_free/hotel_{instance_id}/lambda={lambda_}.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(results)  